In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import mlflow

In [2]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'https://storage.yandexcloud.net'
os.environ['AWS_ACCESS_KEY_ID'] = '33kU43UzyCYfV1jgKUPL'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'WPZnfkNEOlpdZ32hwVGhQ6PNiPPjmFZEajnWUMRe'

In [3]:
mlflow.set_tracking_uri("https://mlflow.lab.karpov.courses")
mlflow.set_experiment(experiment_name = "PySpark-ML-Auto")

In [4]:
spark = SparkSession.builder.appName("PySparkTitanikJob")\
    .config("spark.jars.packages", "org.mlflow:mlflow-spark:1.11.0")\
    .getOrCreate()

:: loading settings :: url = jar:file:/nfs/env/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/karpov/.ivy2/cache
The jars for the packages stored in: /home/karpov/.ivy2/jars
org.mlflow#mlflow-spark added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0003bcbf-3455-4524-8014-5578ae81f838;1.0
	confs: [default]
	found org.mlflow#mlflow-spark;1.11.0 in central
	found org.slf4j#slf4j-api;1.7.25 in central
:: resolution report :: resolve 188ms :: artifacts dl 4ms
	:: modules in use:
	org.mlflow#mlflow-spark;1.11.0 from central in [default]
	org.slf4j#slf4j-api;1.7.25 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retr

In [5]:
spark

In [6]:
mlflow.pyspark.ml.autolog()

In [7]:
titanic_df = spark.read.parquet('train.parquet')

In [8]:
titanic_df.show()

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|
|       0|     3|  male|30.0|    0|    0| 8.4583|       Q|          0|    1|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|          0|    1|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|          4|    0|
|       1|     3|female|27.0|    0|    2|11.1333|       S|          2|    0|
|       1|     2|female|14.0|    1|    0|30.0708|       C|          1|    0|

In [9]:
train, test = titanic_df.randomSplit([0.8, 0.2])

In [10]:
indexer_sex = StringIndexer(inputCol="Sex", outputCol="Sex_index")

In [11]:
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")

In [12]:
feature = VectorAssembler(
    inputCols=["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index"],
    outputCol="features")

In [13]:
rf_classifier = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [14]:
pipeline = Pipeline(stages=[indexer_sex, indexer_embarked, feature, rf_classifier])

In [15]:
p_model = pipeline.fit(train)

2021/11/21 11:34:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9376af1edebf44c1bd6f1372e0c7a276', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
21/11/21 11:34:48 ERROR Instrumentation: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:673)
	at org.apache.spark.ml.util.MLWriter.save

In [16]:
prediction = p_model.transform(test)

In [17]:
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="f1")

In [18]:
p_accuracy = evaluator_accuracy.evaluate(prediction)
p_f1 = evaluator_f1.evaluate(prediction)
print(f"Pipeline model [Accuracy] = {p_accuracy}")
print(f"Pipeline model [F1] = {p_f1}")

Pipeline model [Accuracy] = 0.8315217391304348
Pipeline model [F1] = 0.8286347646396164
